In [1]:
# %%
from pathlib import Path
import os
import re
import pandas as pd
from openpyxl import load_workbook

In [3]:
# 리포지토리 루트 = notebooks의 부모
BASE_DIR = Path.cwd().resolve().parent        # .../전국도시가스보급률
DATA_DIR = BASE_DIR / "data"
OUT_DIR  = BASE_DIR / "out"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SRC      = DATA_DIR / "5. 보급률실적 (1992~2024).xlsx"
TMP_PATH = os.path.splitext(str(SRC))[0] + "_tmp_unmerged.xlsx"

In [4]:
# ──────────────────────────────────────────────────────────────────────────────
# 1) 대상 시트: 기존(1,2,3) + 새 4번째 시트(=index 4)까지
#    ※ 엑셀에서 0-based 인덱스 기준
# ──────────────────────────────────────────────────────────────────────────────
TARGET_SHEETS = [1, 2, 3, 4]

# ──────────────────────────────────────────────────────────────────────────────
# 2) 병합 해제 임시본 저장 (대상 시트만 언머지)
# ──────────────────────────────────────────────────────────────────────────────
wb = load_workbook(SRC)
for sidx in TARGET_SHEETS:
    ws = wb.worksheets[sidx]
    for rng in list(ws.merged_cells.ranges):
        ws.unmerge_cells(str(rng))
wb.save(TMP_PATH)


In [5]:
# ──────────────────────────────────────────────────────────────────────────────
# 3) 시트 → tidy 변환 함수(동적 탐지)
#    - 헤더 구조가 달라도 "세대수/수요가수/보급률" 열 블록을 자동 분류
#    - 연도 행에 2024 한 개만 있어도 처리됨
# ──────────────────────────────────────────────────────────────────────────────
def tidy_from_sheet(sheet_index: int) -> pd.DataFrame:
    raw = pd.read_excel(TMP_PATH, sheet_name=sheet_index, header=None)

    # 기본 위치(0-based). 이전 시트들과 동일하다고 가정
    ROW_LABELS = 2   # "시도별/회사별/공급권역내 세대수/수요가수/보급률" 등의 라벨 행
    ROW_YEARS  = 3   # 연도(들) 행
    ROW_DATA   = 4   # 데이터 시작 행

    COL_SIDO   = 0   # 시도
    COL_COMP   = 1   # 회사

    # 풋노트 제거 (설명행이 시작되는 지점부터 아래를 제거)
    note_pat = re.compile(r"(보급률\s*산정기준|주민등록\s*세대수|공급지역\s*허가|공급권역)", re.I)
    col0 = raw.iloc[:, COL_SIDO].astype(str).fillna("")
    note_rows = col0[col0.str.contains(note_pat)].index
    if len(note_rows) > 0:
        raw = raw.iloc[: note_rows.min(), :]

    # 베이스(시도/회사) 정리
    base = raw.iloc[ROW_DATA:, [COL_SIDO, COL_COMP]].copy()
    base.columns = ["시도", "회사"]
    base["시도"] = base["시도"].ffill()
    base["회사"] = base["회사"].ffill()

    # ── 헤더 라벨/연도 스캔: 병합 해제 후 비어있는 셀을 가로방향 ffill로 채워
    labels_row = raw.iloc[ROW_LABELS, :].astype(str).replace("nan", "")
    labels_row = labels_row.ffill()

    years_row = raw.iloc[ROW_YEARS, :]

    # "연도"로 쓸 수 있는 열(숫자/문자 '2024' 등)만 인덱스 추출
    def parse_year(val):
        try:
            # "2024", 2024.0 등 모두 처리
            y = int(float(str(val).strip()))
            return y
        except Exception:
            return None

    year_by_col = {ci: parse_year(v) for ci, v in years_row.items()}
    year_cols   = [ci for ci, y in year_by_col.items() if y is not None]

    # 레이블로 블록 분류
    def has(text, key):
        return key in str(text)

    sedae_cols = [c for c in year_cols if has(labels_row.iloc[c], "세대")]
    soyo_cols  = [c for c in year_cols if has(labels_row.iloc[c], "수요")]
    rate_cols  = [c for c in year_cols if has(labels_row.iloc[c], "보급률")]

    # 예외: 혹시 레이블이 비어 있으면 위치 기반으로 추정(전통 구조: 세대/수요/보급률 순)
    if not (sedae_cols and soyo_cols and rate_cols):
        # 2개 또는 1개만 잡힐 때를 대비해 좌→우 순서대로 3등분 근사
        if year_cols:
            k = len(year_cols)
            one = max(1, k // 3)
            sedae_cols = sedae_cols or year_cols[:one]
            soyo_cols  = soyo_cols  or year_cols[one:one*2]
            rate_cols  = rate_cols  or year_cols[one*2:]

    # 블록 추출 & 숫자 변환
    def block(cols, years):
        if not cols:
            return pd.DataFrame(index=raw.index[ROW_DATA:], columns=[])
        dfb = raw.iloc[ROW_DATA:, cols].copy()
        dfb.columns = years
        # 숫자화(콤마/공백/% 제거)
        def to_num(s):
            return pd.to_numeric(
                s.astype(str)
                 .str.replace(",", "", regex=False)
                 .str.replace("%", "", regex=False)
                 .str.strip(),
                errors="coerce"
            )
        return dfb.apply(to_num)

    years_sedae = [year_by_col[c] for c in sedae_cols]
    years_soyo  = [year_by_col[c] for c in soyo_cols]
    years_rate  = [year_by_col[c] for c in rate_cols]

    sedae = block(sedae_cols, years_sedae)
    soyo  = block(soyo_cols,  years_soyo)
    rate  = block(rate_cols,  years_rate)

    # long-format으로 변환
    def melt_block(df, value_name):
        if df.shape[1] == 0:
            # 비어 있으면 빈 DF 반환
            return pd.DataFrame(columns=["시도", "회사", "연도", value_name])
        tmp = pd.concat([base.reset_index(drop=True), df.reset_index(drop=True)], axis=1)
        return tmp.melt(id_vars=["시도", "회사"], var_name="연도", value_name=value_name)

    sedae_l = melt_block(sedae, "세대수")
    soyo_l  = melt_block(soyo,  "수요가수")
    rate_l  = melt_block(rate,  "보급률")

    out = (
        sedae_l
        .merge(soyo_l, on=["시도", "회사", "연도"], how="outer")
        .merge(rate_l, on=["시도", "회사", "연도"], how="outer")
    )

    # 합계/계 제거
    drop_patterns = r"(소\s*계|수도권\s*계|지\s*방\s*계|전\s*국\s*계|\(계\))"
    mask = (
        out["시도"].astype(str).str.contains(drop_patterns, na=False)
        | out["회사"].astype(str).str.contains(drop_patterns, na=False)
    )
    out = out[~mask].copy()

    # 정리
    out["연도"] = out["연도"].astype(int)
    out["시도"] = out["시도"].astype(str).str.replace(r"\s+", "", regex=True)
    out["회사"] = out["회사"].astype(str).str.replace(r"\s+", "", regex=True)

    # 보급률은 % 없이 숫자로 저장됨(필요하면 나중에 시각화 시 퍼센트 처리)
    return (
        out[["연도", "시도", "회사", "세대수", "수요가수", "보급률"]]
        .sort_values(["연도", "시도", "회사"])
        .reset_index(drop=True)
    )

In [6]:
# ──────────────────────────────────────────────────────────────────────────────
# 4) 시트별 변환 → 통합
# ──────────────────────────────────────────────────────────────────────────────
dfs = []
for sidx in TARGET_SHEETS:
    try:
        df_i = tidy_from_sheet(sidx)
        if not df_i.empty:
            dfs.append(df_i)
        print(f"sheet {sidx}: rows={len(df_i)}")
    except Exception as e:
        print(f"[WARN] sheet {sidx} 처리 중 오류: {e}")

if not dfs:
    raise RuntimeError("변환된 데이터가 없습니다. 시트/헤더 구조를 확인하세요.")

final = (
    pd.concat(dfs, ignore_index=True)
    .sort_values(["연도", "시도", "회사"])
    .reset_index(drop=True)
)


C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  note_rows = col0[col0.str.contains(note_pat)].index
C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:112: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  out["시도"].astype(str).str.contains(drop_patterns, na=False)
C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:113: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | out["회사"].astype(str).str.contains(drop_patterns, na=False)
C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  note_rows = col0

sheet 1: rows=258
sheet 2: rows=276
sheet 3: rows=276
sheet 4: rows=47


C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:112: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  out["시도"].astype(str).str.contains(drop_patterns, na=False)
C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:113: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | out["회사"].astype(str).str.contains(drop_patterns, na=False)
C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  note_rows = col0[col0.str.contains(note_pat)].index
C:\Users\user\AppData\Local\Temp\ipykernel_75828\3338435881.py:112: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  out["시도"].astyp

In [7]:
# 출력 파일명: 실제 연도 범위로 자동 결정 (예: 2006-2024)
min_y, max_y = int(final["연도"].min()), int(final["연도"].max())
OUT_CSV = OUT_DIR / f"보급률_tidy_({min_y}-{max_y}).csv"
final.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

print(f"✅ 완료: {OUT_CSV} (rows={len(final)})")

✅ 완료: D:\Project\전국도시가스보급률\out\보급률_tidy_(2006-2024).csv (rows=857)
